In [1]:
import pandas as pd
import numpy as np
from qiskit import QuantumCircuit
from qiskit import Aer, execute
from qiskit.quantum_info import Statevector

### Lectura del fichero CSV
* Se importa el archivo csv.
* se eliminan las columnas que no son datos y convierte el dataset a binario
* Se separa el dataset en dos, filtrando por train y test

In [2]:
data = pd.read_csv('Datos_Montabilidad_Anom_447-6-1.csv',delimiter=';')

# Filtrado y construcción del dataframe binario
df_binario = data.iloc[:, 2:-2].apply(lambda x: x > 0).astype(int)

In [3]:
#Queremos evaluar colummnas, no filas
#Tomamos de ejemplo el sensro1 y el sensor2



import math
x = df_binario[data['Set'] == 'Train'].head(1).to_numpy()[0]
y = df_binario[data['Set'] == 'Test'].head(1).to_numpy()[0]

x_extended = np.pad(x, (0, 1024 - len(x)), 'constant')
y_extended = np.pad(y, (0, 1024 - len(y)), 'constant')

numero_componentes = len(x_extended)
nqubits = math.ceil(np.log2(numero_componentes))    # compute how many qubits needed to encode either x or y

circ = QuantumCircuit(nqubits+1)   # create circuit lo qubits de entrada + el qubit de ansatz
vec = np.concatenate((x_extended,y_extended))   # concatenate x and y as above, with renormalisation

circ.initialize(vec, range(nqubits+1),normalize=True)
circ.h(nqubits)                    # apply hadamard to bottom qubit

state = Statevector(circ)

In [4]:
backend = Aer.get_backend('statevector_simulator')
job = execute(circ, backend=backend)

result = job.result()
o = np.real(result.get_statevector(circ))
#print(result.get_statevector(circ))
#print(o)

m_sum = 0
for l in range(numero_componentes):
    m_sum += o[l]**2 #𝑃(0)

producto_interno = 2*m_sum-1 #𝑅𝑒[⟨𝑥||𝑦⟩]=2𝑃(0)−1
print(producto_interno)

0.3109243697478974
